# Game Utils
This file contains functions which are used in the game implementation.

In [ ]:
# Test States
s = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))
s2 = ((0, 0), (
    ('w', 'w', ' ', 'b', 'w', 'w', 'w', 'w'),
    ('b', 'w', ' ', 'b', 'w', ' ', 'b', 'w'),
    ('b', 'b', 'b', 'b', 'w', ' ', 'b', 'w'),
))
s3 = ((7, 7), (
    ('w', 'w', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', 'b', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', ' ')
))
s4 = ((0, 0), (
    ('b', 'b', 'b', ' ', ' ', 'w', ' ', ' '),
    (' ', 'w', ' ', 'b', ' ', 'w', ' ', ' '),
    (' ', 'b', ' ', 'w', ' ', 'w', ' ', ' ')
))
s5 = ((0, 0), (
    ('b', 'b', ' ', ' ', ' ', 'w', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', 'b', ' ', 'w'),
    (' ', ' ', ' ', ' ', ' ', 'w', ' ', ' ')
))
s6 = ((5, 5), (
    ('b', ' ', 'b', 'w', ' ', ' ', ' ', ' '),
    (' ', 'b', ' ', 'w', ' ', ' ', ' ', ' '),
    (' ', 'b', ' ', 'w', ' ', ' ', ' ', ' ')
))

## Player

In [ ]:
# Calculates the opponent of the given player
def opponent(p):
    return 'b' if p == 'w' else 'w'

## Stones

In [ ]:
# Calculates wether or not a player still has stones which he has not placed yet
def hasPlaceableStones(s, p):
    # Extract the count of the placeable stones for black and white
    ((cw, cb), _) = s
    # Return wether or not the given player has at least one stone
    return cw >= 1 if p == 'w' else cb >= 1

In [ ]:
# Counts how many stones the given player has left on the board
def countStones(s, p):
    # Extract the board from the state
    (_, board) = s
    # Count how many times player occurs on the board
    return [cell for ring in board for cell in ring].count(p)

In [ ]:
# Calculates wether or not the player is allowed to jump with his stones,
# instead of just moving them
def isAllowedToJump(s, p):
    return countStones(s, p) <= 3

In [ ]:
# Calculates if the player has stones left to place or enought (at least 3) stones to continue playing
def hasEnoughStones(s, p):
    return hasPlaceableStones(s, p) or countStones(s, p) >= 3

In [ ]:
# Returns the player on the given coord
def getPlayerAt(board, coord):
    (r, c) = coord
    return board[r][c]

In [ ]:
# Removes a stone from the player's stash and returns the new state
def removeFromStash(s, p):
    return ((s[0][0] - (1 if p == 'w' else 0), s[0][1] - (1 if p == 'b' else 0)), s[1])

## Cells

In [ ]:
# Returns a set of tuples containing all coordinates of the cells owned by p
# Set of Tuples(ring, cell)
def findCellsOf(board, p):
    # Iterate over all cells and select only empty cells
    return {(r, c) for r in range(0, 3) for c in range(0, 8) if board[r][c] == p}

In [ ]:
# Returns a set of tuples containing all coordinates of the empty cells
# Set of Tuples(ring, cell)
def findEmptyCells(board):
    # Empty cells are marked as ' '
    return findCellsOf(board, ' ')

In [ ]:
# Returns a set of tuples containing all coordinates of the empty cells neighboring the given cell
# Set of Tuples(ring, cell)
def findNeighboringEmptyCells(board, coordinates):
    (rootR, rootC) = coordinates
    return {(r, c) for (r, c) in findEmptyCells(board) if (r == rootR and (c == (rootC + 7) % 8 or c == (rootC + 1) % 8)) 
                                                        or (c % 2 == 1 and c == rootC and (r == rootR - 1 or r == rootR + 1)) }


In [ ]:
def place(board, coordinates, player):
    (r, c) = coordinates
    return tuple(
        tuple(
            player if (c == ic) and (r == ir) else board[ir][ic]
            for ic in range(0, 8)
        ) for ir in range(0, 3)
    )

## Muehlen

In [ ]:
# Returns the coordinates of all Muehlen the given player has
# Set of Frozensets of Tuples(ring, cell)
def findMuehlen(board, p):
    # Calculate all Muehlen on the rings
    return {
        frozenset((r, (c+o)%8) for o in range(0, 3))
        # Iterate over all rings
        for r in range(0, 3)
        # Iterate over all corners
        for c in range(0, 8, 2)
        # All 3 following cells starting at the given corner have to belong to the player
        if all(
            cell == p
            # Iterate over all 3 cells of the given side (c) by rotating them to the top left corner
            # (the beginning of the list) and then checking the first 3 cells
            for cell in (board[r][c:] + board[r][:c])[0:3]
        )
    # Calculate all Muehlen crossing the rings
    } | {
        frozenset((r, c) for r in range(0, 3))
        # Iterate over cells in the middle of a side
        for c in range(1, 8, 2)
        # All 3 cells in the middle of a given side have to belong to the player
        if all(
            board[r][c] == p
            # Iterate over all 3 rings
            for r in range(0, 3)
        )
    }

In [ ]:
# Returns the number of Muehlen the player build in his turn.
# The board is the board after his turn, but bevor he executed his Muehlen.
def countNewMuehlen(board, oldMuehlen, p):
    return len(findMuehlen(board, p) - oldMuehlen)

In [ ]:
# Returns all possible boards after the player has executed his allowed Muehle
# Executing a Muehle means removing an opponent stone
def executeMuehle(board, p):
    # Compute the Muehlen of the opponent as these stones can not be removed
    opponentMuehlen = {
        cell
        for muehle in findMuehlen(board, opponent(p))
        for cell in muehle
    }
    opponentCells = findCellsOf(board, opponent(p))

    # Except when there are no other stones than the stones in the Muehlen
    if len(opponentCells - opponentMuehlen) == 0:
        opponentMuehlen = set()

    # Compute all boards with an opponent stone removed
    return {
        place(board, cell, ' ')
        for cell in opponentCells
        if cell not in opponentMuehlen
    }